In [1]:
#Import necessary packages
import itertools as it
import time
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
start_time = time.time()

#Enter game table and change the name of the game table you want to run to "table"
table = [[[3],[2],[4]],[[0],[3],[1]],[[4],[1],[2]],[[1],[0],[3]]]

squareAntiprism = [[[2],[0],[0],[4],[8],[0],[0],[0],[5],[0]],
        [[3],[7],[0],[0],[1],[8],[0],[0],[0],[0]],
        [[4],[2],[6],[0],[0],[0],[7],[0],[0],[0]],
        [[1],[0],[3],[5],[0],[0],[0],[6],[0],[0]],
        [[0],[0],[0],[1],[0],[0],[0],[4],[8],[6]],
        [[0],[0],[4],[0],[0],[0],[3],[5],[0],[7]],
        [[0],[3],[0],[0],[0],[2],[6],[0],[0],[8]],
        [[0],[0],[0],[0],[2],[7],[0],[0],[1],[5]]]

smallerGrid = [[[2],[0],[0],[0],[4]],
        [[5],[3],[0],[0],[1]],
        [[0],[6],[0],[0],[2]],
        [[1],[0],[5],[0],[7]],
        [[4],[2],[8],[6],[0]],
        [[0],[5],[0],[9],[3]],
        [[0],[0],[4],[0],[8]],
        [[0],[0],[7],[5],[9]],
        [[0],[0],[0],[8],[6]]]

largerGrid = [[[2],[0],[0],[0],[0],[4]],
        [[5],[3],[0],[0],[0],[1]],
        [[0],[6],[0],[0],[10],[2]],
        [[1],[0],[5],[0],[0],[7]],
        [[4],[2],[8],[6],[0],[0]],
        [[0],[5],[0],[9],[3],[11]],
        [[0],[0],[4],[0],[0],[8]],
        [[0],[0],[7],[5],[0],[9]],
        [[0],[0],[0],[8],[0],[6]],
        [[0],[0],[0],[0],[11],[3]],
        [[0],[0],[0],[0],[6],[10]]]

posterBoard = [[[5],[0],[2],[4]],[[0],[0],[3],[1]],[[0],[0],[4],[2]],[[1],[0],[5],[3]],
         [[4],[6],[1,7],[0]],[[0],[7],[5],[0]],[[0],[5],[6],[0]]]

adaptedPosterBoard = [[[5],[0],[2],[4]],
        [[0],[0],[8],[1]],
        [[0],[0],[4],[8]],
        [[1],[0],[5],[3]],
        [[4],[6],[1,7],[0]],
        [[0],[7],[5],[0]],
        [[0],[5],[6],[0]],
        [[0],[0],[3],[2]]]

doubleTriangle = [[[5],[7],[9],[2,6,8]],[[1],[0],[0],[3]],[[2],[0],[0],[4]],
                  [[3],[0],[0],[5]],[[4],[0],[0],[1]],[[0],[1],[0],[7]],
                  [[0],[6],[0],[1]],[[0],[0],[1],[9]],[[0],[0],[8],[1]]]

OGtable = [[[5],[7],[2,6]],[[1],[0],[3]],[[2],[0],[4]],[[3],[0],[5]],
                    [[4],[0],[1]],[[0],[1],[7]],[[0],[6],[1]]]

In [3]:
#Convert table to zero-based numbering
for i in range(len(table)):
    for j in range(len(table[i])):
        for k in range(len(table[i][j])):
            table[i][j][k] -= 1

#Create necessary lists and dictionaries
mapping = {}
reverseMapping = {}

boards = [[]]
illegal = []
cycles = []

emptyBoard = []
digraph = nx.DiGraph()

In [4]:
#Get moves from the table by iterating through values in the table
for i in range(len(table)):
    for j in range(len(table[i])):
        for k in range(len(table[i][j])):
            value = table[i][j][k]
        
            startPoint = (i,j,k)
            
            #Consider value if it represents an edge
            if value != -1:
                
                #Find the pair of the initial value
                for u in range(len(table[value])):
                    for v in range(len(table[value][u])):
                        if table[value][u][v] == i:
                            endPoint = (value, u, v)

                #Check if the endPoint already has its own mapping
                if endPoint not in mapping.keys():
                    mapping[startPoint] = endPoint
                    reverseMapping[endPoint] = startPoint
                    boards.append([])
                    emptyBoard.append(0)

#Get sinks/sources from the table
for i in range(len(table)):
    source = emptyBoard.copy()
    sink = emptyBoard.copy()
    
    #Set all moves to be the same direction with respect to the row vertex
    for j in range(len(table[i])):
        for k in range(len(table[i][j])):
            if table[i][j][k] > -1:
                
                #Set value as positive or negative based on the orientation of the edge in "mapping"
                if (i,j,k) in mapping.keys():
                    source[list(mapping.keys()).index((i,j,k))] = 1
                    sink[list(mapping.keys()).index((i,j,k))] = -1
                else:
                    source[list(reverseMapping.keys()).index((i,j,k))] = -1
                    sink[list(reverseMapping.keys()).index((i,j,k))] = 1
                    
    #Add to list of sinks and sources
    illegal.append(source)
    illegal.append(sink)

#Get cycles from the table
for j in range(len(table[0])-1):
    clockwise = emptyBoard.copy()
    counterClockwise = emptyBoard.copy()
    
    #Set all moves to be the same direction with respect to the cell
    for i in range(len(table)):
        for k in range(len(table[i][j])):
            if table[i][j][k] > -1:
                
                #Set value as positive or negative based on the orientation of the edge in "mapping"
                if (i,j,k) in mapping.keys():
                    clockwise[list(mapping.keys()).index((i,j,k))] = 1
                    counterClockwise[list(mapping.keys()).index((i,j,k))] = -1
                else:
                    clockwise[list(reverseMapping.keys()).index((i,j,k))] = -1
                    counterClockwise[list(reverseMapping.keys()).index((i,j,k))] = 1
                    
    #Add to list of cycles
    cycles.append(clockwise)
    cycles.append(counterClockwise)

In [5]:
#Consider all possible boards by creating a list that is the length of the number of edges in 
# the table and considering all permutations with repetition where a 0 represents the edge being 
# unmarked, a 1 from the first entry in the mapping to the second, and a -1 in the oppposite direction
num = 1
for combination in it.product([0,1,-1],repeat = len(mapping)):
    
    #Program running update
    if num % 1000000 == 0:
        print(num, ": {:.3f} seconds".format(time.time() - start_time), sep = "")
    
    #Check for sinks/sources by checking if board is equal to all nonzero entries of a sink/source
    legal = True
    for sink in illegal:
        equivalent = True
        for i in range(len(combination)):
            if (sink[i] != 0) and (sink[i] != combination[i]):
                equivalent = False
        if equivalent == True:
            legal = False
            
    #Check for cycles by checking if board is equal to all nonzero entries of a cycle
    unfinished = True
    for cycle in cycles:
        equivalent = True
        for i in range(len(combination)):
            if (cycle[i] != 0) and (cycle[i] != combination[i]):
                equivalent = False
        if equivalent == True:
            unfinished = False
    
    #Add legal combinations to boards and digraph
    if legal:
        marked = 0
        
        #Add to proper part of boards list given number of moves played in the board
        for move in combination:
            if move != 0:
                marked += 1
        
        boards[marked].append(combination)
        
        #Label as an end state if a cycle exists
        if unfinished:
            digraph.add_node(combination, position = "", layer = marked)
        else:
            digraph.add_node(combination, position = "p", layer = marked)
    num += 1

print("End Time: ", " {:.3f} seconds".format(time.time() - start_time))
print()

#Calculate how many legal boards there are
sum = 0
for i in range(len(boards)):
    sum += len(boards[i])
print("Total legal boards:",sum)

End Time:   0.072 seconds

Total legal boards: 133


In [6]:
#Add edges to digraph by checking every node
num = 0
for i in range(len(mapping)):
    for board in boards[i]:
        
        #Program running update
        num += 1
        if num % 1000 == 0:
            print(num, ": {:.3f} seconds".format(time.time() - start_time), sep = "")
        
        #Only add edges to boards without cycles
        if digraph.nodes[board]["position"] == "":

            #For every zero in the list, add a 1 or -1 and add an edge to that node if it exists
            for j in range(len(board)):
                if board[j] == 0:
                    positive = list(board)
                    negative = list(board)
                    positive[j] = 1
                    negative[j] = -1
                    if tuple(positive) in digraph.nodes:
                        digraph.add_edge(board,tuple(positive))
                    if tuple(negative) in digraph.nodes:
                        digraph.add_edge(board,tuple(negative))
                        
print("Time: ", " {:.3f} seconds".format(time.time() - start_time))

Time:   0.093 seconds


In [7]:
#Label the nodes in the digraph as n and p positions starting at the end of the game
num = 0
for i in range(len(boards)):
    print(i)
    for board in boards[len(boards) - i - 1]:
        
        #Program running update
        num += 1
        if num % 1000000 == 0:
            print(num, ": {:.3f} seconds".format(time.time() - start_time), sep = "")
        
        #Remove nodes with an in-degree of zero because they cannot be reached without the game already ending
        if digraph.in_degree(board) == 0 and i != len(boards) - 1:
            turns = 0
            for entry in board:
                if entry != 0:
                    turns += 1
                    
            digraph.remove_node(board)
        
        #P-position if an end state
        elif len(digraph.edges(board)) == 0:
            digraph.nodes[board]["position"] = "p"
            
        else:
            #N-position if directed to a p-position
            for edge in digraph.edges(board):
                if digraph.nodes[edge[1]]["position"] == "p":
                    digraph.nodes[board]["position"] = "n"

            #P-position if only directed to N-positions
            if digraph.nodes[board]["position"] == "":
                digraph.nodes[board]["position"] = "p"

print("Time: ", " {:.3f} seconds".format(time.time() - start_time))

0
1
2
3
4
5
Time:   0.136 seconds


In [8]:
#Print possible moves based on board
for edge in digraph.edges(tuple(emptyBoard)):
    print(edge, "\t", digraph.nodes[edge[1]]["position"])
    
print()
print("Minimum Time: ", " {:.3f} seconds".format(time.time() - start_time))

((0, 0, 0, 0, 0), (1, 0, 0, 0, 0)) 	 p
((0, 0, 0, 0, 0), (-1, 0, 0, 0, 0)) 	 p
((0, 0, 0, 0, 0), (0, 1, 0, 0, 0)) 	 n
((0, 0, 0, 0, 0), (0, -1, 0, 0, 0)) 	 n
((0, 0, 0, 0, 0), (0, 0, 1, 0, 0)) 	 n
((0, 0, 0, 0, 0), (0, 0, -1, 0, 0)) 	 n
((0, 0, 0, 0, 0), (0, 0, 0, 1, 0)) 	 n
((0, 0, 0, 0, 0), (0, 0, 0, -1, 0)) 	 n
((0, 0, 0, 0, 0), (0, 0, 0, 0, 1)) 	 n
((0, 0, 0, 0, 0), (0, 0, 0, 0, -1)) 	 n

Minimum Time:   0.155 seconds


In [9]:
#Calculate maximum number of unmarkable edges by counting unmarkable edges in all terminal states
maxUnmarkable = 0
for node in digraph.nodes:
    
    #Count unmarkable edges of terminal states
    if digraph.out_degree(node) == 0:
        unmarkable = 0
        for i in range(len(mapping)):
            if node[i] == 0:
                unmarkable += 1
                
        #Change maxUnmarkable if greater
        if unmarkable > maxUnmarkable:
            maxUnmarkable = unmarkable
print(maxUnmarkable)

2


In [10]:
ns = [[],[],[],[],[],[],[],[],[],[]]
for node in digraph.nodes:
    if digraph.nodes[node]["position"] == "n":
        onlyP = True
        marked = 0
        for edge in digraph.edges(node):
            if digraph.nodes[edge[1]]["position"] == "n":
                onlyP = False

        if onlyP == False:
            for item in node:
                if item != 0:
                    marked += 1

            ns[marked].append(node)

for i in range(10):
    print(ns[i])
    print(len(ns[i]))
    print()

[(0, 0, 0, 0, 0)]
1

[(0, 0, 0, 0, 1), (0, 0, 0, 0, -1), (0, 0, 0, 1, 0), (0, 0, 0, -1, 0), (0, 0, 1, 0, 0), (0, 0, -1, 0, 0), (0, 1, 0, 0, 0), (0, -1, 0, 0, 0)]
8

[(0, 0, 1, 0, -1), (0, 0, 1, 1, 0), (0, 0, -1, 0, 1), (0, 0, -1, -1, 0), (0, 1, 0, 0, -1), (0, 1, 0, 1, 0), (0, -1, 0, 0, 1), (0, -1, 0, -1, 0), (1, 0, 0, 0, 1), (1, 0, 0, 0, -1), (1, 0, 0, 1, 0), (1, 0, 0, -1, 0), (1, 0, 1, 0, 0), (1, 0, -1, 0, 0), (1, 1, 0, 0, 0), (1, -1, 0, 0, 0), (-1, 0, 0, 0, 1), (-1, 0, 0, 0, -1), (-1, 0, 0, 1, 0), (-1, 0, 0, -1, 0), (-1, 0, 1, 0, 0), (-1, 0, -1, 0, 0), (-1, 1, 0, 0, 0), (-1, -1, 0, 0, 0)]
24

[(0, 0, 1, 1, -1), (0, 0, 1, -1, -1), (0, 0, -1, 1, 1), (0, 0, -1, -1, 1), (0, 1, 0, 1, 1), (0, 1, 0, 1, -1), (0, 1, 1, 0, -1), (0, 1, 1, 1, 0), (0, 1, -1, 0, 1), (0, 1, -1, 1, 0), (0, -1, 0, -1, 1), (0, -1, 0, -1, -1), (0, -1, 1, 0, -1), (0, -1, 1, -1, 0), (0, -1, -1, 0, 1), (0, -1, -1, -1, 0), (1, 0, 0, 1, 1), (1, 0, 0, -1, -1), (1, 0, 1, -1, 0), (1, 0, -1, 1, 0), (1, 1, 0, 0, 1), (1, 1, -1, 0

In [11]:
#Output image of directed graph
# matplotlib qt

#Open drawing, set colors, and determine board parameters
drawing = nx.multipartite_layout(digraph, subset_key="layer")
color_map = [(255/255, 184/255, 28/255) if digraph.nodes[node]["position"] == "p" else (65/255, 65/255, 65/255) for node in digraph] 
nx.draw_networkx(digraph, pos=drawing, node_size = 50, with_labels=False, node_color= color_map)

#Save figure as pdf and show on screen
plt.savefig('digraph.pdf')
plt.figure(figsize=(12,12))
plt.rcParams['figure.figsize'] = [12,12]